## Imports

In [1]:
from lxml import html
from lxml import etree
import requests
import os
import math
import numpy as np
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.utils as torch_img_utils
from torch.utils.data import DataLoader
import torchvision.models as models
import torch.backends.cudnn as cudnn
import torchvision
import torch.autograd as autograd
from PIL import Image
import imp
import sys
import math
import time
import random
import shutil
import cv2
import scipy.misc
from glob import glob

from tqdm import tqdm
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
plt.style.use('bmh')

%matplotlib inline

In [2]:
# https://docs.python.org/3/library/html.parser.html
# Elements https://developer.mozilla.org/en-US/docs/Web/HTML/Element
# https://doc.scrapy.org/en/latest/topics/link-extractors.html
# https://doc.scrapy.org/en/latest/topics/spiders.html#scrapy.spiders.CrawlSpider

## Boostrap Classes

In [3]:
BOOTSTRAP_CLASSES = {'navbar' : 1,
'caret' : 1,
'label' : 1,
'table' : 1,
'img-responsive' : 1,
'img-rounded' : 1,
'img-thumbnail' : 1,
'img-circle' : 1,
'sr-only' : 1,
'lead' : 1,
'text-muted' : 1,
'text-primary' : 1,
'text-warning' : 1,
'text-danger' : 1,
'text-success' : 1,
'text-info' : 1,
'text-left' : 1,
'text-right' : 1,
'text-center' : 1,
'h6' : 1,
'h1' : 1,
'h2' : 1,
'h3' : 1,
'h4' : 1,
'h5' : 1,
'page-header' : 1,
'list-unstyled' : 1,
'list-inline' : 1,
'initialism' : 1,
'pull-right' : 1,
'prettyprint' : 1,
'pre-scrollable' : 1,
'container' : 1,
'row' : 1,
'col-lg-12' : 1,
'col-xs-11' : 1,
'col-xs-1' : 1,
'col-xs-2' : 1,
'col-xs-3' : 1,
'col-xs-4' : 1,
'col-xs-5' : 1,
'col-xs-6' : 1,
'col-xs-7' : 1,
'col-xs-8' : 1,
'col-xs-9' : 1,
'col-xs-10' : 1,
'col-xs-12' : 1,
'col-sm-11' : 1,
'col-sm-1' : 1,
'col-sm-2' : 1,
'col-sm-3' : 1,
'col-sm-4' : 1,
'col-sm-5' : 1,
'col-sm-6' : 1,
'col-sm-7' : 1,
'col-sm-8' : 1,
'col-sm-9' : 1,
'col-sm-10' : 1,
'col-sm-12' : 1,
'col-sm-push-1' : 1,
'col-sm-push-2' : 1,
'col-sm-push-3' : 1,
'col-sm-push-4' : 1,
'col-sm-push-5' : 1,
'col-sm-push-6' : 1,
'col-sm-push-7' : 1,
'col-sm-push-8' : 1,
'col-sm-push-9' : 1,
'col-sm-push-10' : 1,
'col-sm-push-11' : 1,
'col-sm-pull-1' : 1,
'col-sm-pull-2' : 1,
'col-sm-pull-3' : 1,
'col-sm-pull-4' : 1,
'col-sm-pull-5' : 1,
'col-sm-pull-6' : 1,
'col-sm-pull-7' : 1,
'col-sm-pull-8' : 1,
'col-sm-pull-9' : 1,
'col-sm-pull-10' : 1,
'col-sm-pull-11' : 1,
'col-sm-offset-1' : 1,
'col-sm-offset-2' : 1,
'col-sm-offset-3' : 1,
'col-sm-offset-4' : 1,
'col-sm-offset-5' : 1,
'col-sm-offset-6' : 1,
'col-sm-offset-7' : 1,
'col-sm-offset-8' : 1,
'col-sm-offset-9' : 1,
'col-sm-offset-10' : 1,
'col-sm-offset-11' : 1,
'col-md-11' : 1,
'col-md-1' : 1,
'col-md-2' : 1,
'col-md-3' : 1,
'col-md-4' : 1,
'col-md-5' : 1,
'col-md-6' : 1,
'col-md-7' : 1,
'col-md-8' : 1,
'col-md-9' : 1,
'col-md-10' : 1,
'col-md-12' : 1,
'col-md-push-0' : 1,
'col-md-push-1' : 1,
'col-md-push-2' : 1,
'col-md-push-3' : 1,
'col-md-push-4' : 1,
'col-md-push-5' : 1,
'col-md-push-6' : 1,
'col-md-push-7' : 1,
'col-md-push-8' : 1,
'col-md-push-9' : 1,
'col-md-push-10' : 1,
'col-md-push-11' : 1,
'col-md-pull-0' : 1,
'col-md-pull-1' : 1,
'col-md-pull-2' : 1,
'col-md-pull-3' : 1,
'col-md-pull-4' : 1,
'col-md-pull-5' : 1,
'col-md-pull-6' : 1,
'col-md-pull-7' : 1,
'col-md-pull-8' : 1,
'col-md-pull-9' : 1,
'col-md-pull-10' : 1,
'col-md-pull-11' : 1,
'col-md-offset-0' : 1,
'col-md-offset-1' : 1,
'col-md-offset-2' : 1,
'col-md-offset-3' : 1,
'col-md-offset-4' : 1,
'col-md-offset-5' : 1,
'col-md-offset-6' : 1,
'col-md-offset-7' : 1,
'col-md-offset-8' : 1,
'col-md-offset-9' : 1,
'col-md-offset-10' : 1,
'col-md-offset-11' : 1,
'col-lg-11' : 1,
'col-lg-1' : 1,
'col-lg-2' : 1,
'col-lg-3' : 1,
'col-lg-4' : 1,
'col-lg-5' : 1,
'col-lg-6' : 1,
'col-lg-7' : 1,
'col-lg-8' : 1,
'col-lg-9' : 1,
'col-lg-10' : 1,
'col-lg-push-0' : 1,
'col-lg-push-1' : 1,
'col-lg-push-2' : 1,
'col-lg-push-3' : 1,
'col-lg-push-4' : 1,
'col-lg-push-5' : 1,
'col-lg-push-6' : 1,
'col-lg-push-7' : 1,
'col-lg-push-8' : 1,
'col-lg-push-9' : 1,
'col-lg-push-10' : 1,
'col-lg-push-11' : 1,
'col-lg-pull-0' : 1,
'col-lg-pull-1' : 1,
'col-lg-pull-2' : 1,
'col-lg-pull-3' : 1,
'col-lg-pull-4' : 1,
'col-lg-pull-5' : 1,
'col-lg-pull-6' : 1,
'col-lg-pull-7' : 1,
'col-lg-pull-8' : 1,
'col-lg-pull-9' : 1,
'col-lg-pull-10' : 1,
'col-lg-pull-11' : 1,
'col-lg-offset-0' : 1,
'col-lg-offset-1' : 1,
'col-lg-offset-2' : 1,
'col-lg-offset-3' : 1,
'col-lg-offset-4' : 1,
'col-lg-offset-5' : 1,
'col-lg-offset-6' : 1,
'col-lg-offset-7' : 1,
'col-lg-offset-8' : 1,
'col-lg-offset-9' : 1,
'col-lg-offset-10' : 1,
'col-lg-offset-11' : 1,
'table-bordered' : 1,
'table-responsive' : 1,
'form-control' : 1,
'form-group' : 1,
'checkbox' : 1,
'checkbox-inline' : 1,
'input-sm' : 1,
'input-lg' : 1,
'control-label' : 1,
'input-group-addon' : 1,
'form-control-static' : 1,
'help-block' : 1,
'btn' : 1,
'active' : 1,
'btn-default' : 1,
'btn-primary' : 1,
'btn-warning' : 1,
'btn-danger' : 1,
'btn-success' : 1,
'btn-info' : 1,
'btn-link' : 1,
'btn-lg' : 1,
'btn-xs' : 1,
'btn-block' : 1,
'fade' : 1,
'in' : 1,
'collapse' : 1,
'collapsing' : 1,
'glyphicon' : 1,
'dropdown' : 1,
'dropdown-menu' : 1,
'divider' : 1,
'dropdown-header' : 1,
'dropdown-backdrop' : 1,
'btn-group-vertical' : 1,
'btn-group' : 1,
'dropdown-toggle' : 1,
'btn-group-justified' : 1,
'input-group' : 1,
'col' : 1,
'input-group-btn' : 1,
'nav' : 1,
'nav-divider' : 1,
'nav-tabs' : 1,
'nav-justified' : 1,
'nav-tabs-justified' : 1,
'pill-pane' : 1,
'navbar-header' : 1,
'navbar-collapse' : 1,
'navbar-static-top' : 1,
'navbar-fixed-bottom' : 1,
'navbar-fixed-top' : 1,
'navbar-brand' : 1,
'navbar-toggle' : 1,
'icon-bar' : 1,
'navbar-nav' : 1,
'navbar-left' : 1,
'navbar-right' : 1,
'navbar-form' : 1,
'navbar-btn' : 1,
'navbar-text' : 1,
'navbar-default' : 1,
'navbar-link' : 1,
'navbar-inverse' : 1,
'breadcrumb' : 1,
'pagination' : 1,
'pager' : 1,
'label-default' : 1,
'label-primary' : 1,
'label-success' : 1,
'label-info' : 1,
'label-warning' : 1,
'label-danger' : 1,
'badge' : 1,
'jumbotron' : 1,
'thumbnail' : 1,
'caption' : 1,
'alert' : 1,
'alert-link' : 1,
'alert-dismissable' : 1,
'close' : 1,
'alert-success' : 1,
'alert-info' : 1,
'alert-warning' : 1,
'alert-danger' : 1,
'progress' : 1,
'progress-bar' : 1,
'progress-bar-success' : 1,
'progress-bar-info' : 1,
'progress-bar-warning' : 1,
'progress-bar-danger' : 1,
'media-body' : 1,
'media' : 1,
'media-object' : 1,
'media-heading' : 1,
'pull-left' : 1,
'media-list' : 1,
'list-group' : 1,
'list-group-item' : 1,
'list-group-item-heading' : 1,
'list-group-item-text' : 1,
'panel' : 1,
'panel-body' : 1,
'panel-heading' : 1,
'panel-title' : 1,
'panel-footer' : 1,
'panel-default' : 1,
'panel-primary' : 1,
'panel-success' : 1,
'panel-warning' : 1,
'panel-danger' : 1,
'panel-info' : 1,
'well' : 1,
'well-lg' : 1,
'well-sm' : 1,
'modal-open' : 1,
'modal' : 1,
'modal-dialog' : 1,
'modal-content' : 1,
'modal-backdrop' : 1,
'modal-header' : 1,
'modal-title' : 1,
'modal-body' : 1,
'modal-footer' : 1,
'tooltip' : 1,
'top' : 1,
'right' : 1,
'bottom' : 1,
'left' : 1,
'tooltip-inner' : 1,
'tooltip-arrow' : 1,
'popover' : 1,
'popover-title' : 1,
'popover-content' : 1,
'arrow' : 1,
'carousel' : 1,
'carousel-inner' : 1,
'item' : 1,
'prev' : 1,
'next' : 1,
'carousel-control' : 1,
'glyphicon-chevron-right' : 1,
'icon-next' : 1,
'carousel-indicators' : 1,
'carousel-caption' : 1,
'hide' : 1,
'show' : 1,
'invisible' : 1,
'text-hide' : 1,
'affix' : 1,
'hidden' : 1,
'visible-xs' : 1,
'visible-sm' : 1,
'visible-md' : 1,
'visible-lg' : 1,
'hidden-xs' : 1,
'hidden-sm' : 1,
'hidden-md' : 1,
'hidden-lg' : 1,
'visible-print' : 1,
'hidden-print' : 1}

## Parser

In [10]:
#Remove all script tags
from html.parser import HTMLParser

def extract_first_class_from_attrs(attrs):
    for a in attrs:
        if a[0] == 'class':
            return a[1]
    return None

class MyHTMLParser(HTMLParser):
    def __init__(self):
        super(MyHTMLParser, self).__init__()
        self.blocks = []

    def handle_starttag(self, tag, attrs):
        cssclass = extract_first_class_from_attrs(attrs)
        if cssclass and cssclass in BOOTSTRAP_CLASSES:
            self.blocks.append('<'+tag+' class="%s">' % cssclass)
        else:
            self.blocks.append('<'+tag+'>')

    def handle_endtag(self, tag):
        #print("Encountered an end tag :", tag)
        self.blocks.append('</'+tag+'>')

    def handle_data(self, data):
        #print("Encountered some data  :", data)
        self.blocks.append('CONTENT')
        
    def get_blocks(self):
        return self.blocks

# page1 = requests.get('http://getbootstrap.com/getting-started/').text
# tags = extract_tags(page1)
# join_tags = ''.join(tags)
# print(join_tags)
    
def extract_tags(html_text):
    parser = MyHTMLParser()
    parser.feed(html_text)
    return parser.get_blocks()

def get_paths_to_files(dir_path):
    filepaths = []
    fnames = []
    for (dirpath, dirnames, filenames) in os.walk(dir_path):
        filepaths.extend(os.path.join(dirpath, f) for f in filenames if not f[0] == '.')
        fnames.extend([f for f in filenames if not f[0] == '.'])
    return filepaths, fnames

def extract_tags_from_file(fpath):
    with open(fpath,'r') as f:
        text = f.read()
    tags = extract_tags(text)
    return tags

def get_unique_tags_in_dir(dir_path):
    fpaths,fnames = get_paths_to_files(dir_path)
    unique_tags = {}
    for f in fpaths:
        tags = extract_tags_from_file(f)
        for t in tags:
            if t not in unique_tags:
                unique_tags[t] = None
    return list(unique_tags.keys())

In [11]:
PAGES_PATH='/home/bfortuner/workplace/websitegan/websitegan/pages'
fpaths,fnames = get_paths_to_files(PAGES_PATH)
#extract_tags_from_file(fpaths[0])
UNIQUE_TAGS = get_unique_tags_in_dir(PAGES_PATH)

## Embeddings

In [14]:
def tags_to_embds(tags, tags_to_embds):
    return [tags_to_embds[t] for t in tags]

def embds_to_tags(embds, embds_to_tags):
    return [embds_to_tags[e] for e in embds]

def get_tag_embeddings(tags):
    tag_to_embd = {}
    for t in tags:
        embd = sum([ord(c) for c in list(t)])
        tag_to_embd[t] = embd
    return tag_to_embd

def normalize_embds(tag_to_embd):
    tag_to_embd_copy = dict(tag_to_embd)
    embeddings = list(tag_to_embd_copy.values())
    mean = np.mean(embeddings)
    std = np.std(embeddings)
    for t,e in tag_to_embd_copy.items():
        tag_to_embd_copy[t] = (e-mean)/std
    return tag_to_embd_copy

def get_embeddings_from_dir(dir_path, tags_to_embds):
    fpaths,fnames = get_paths_to_files(dir_path)
    embeddings = []
    for f in fpaths:
        tags = extract_tags_from_file(f)
        embds = tags_to_embds(tags, tags_to_embds)
        embeddings.append(embds)
    return embeddings

def build_webpage_from_tags(tags):
    return ''.join(tags)

def build_webpage_from_embd_tensor(embd_tnsr):
    #### TODO ####
    ''.join(tags)

In [15]:
TAG_TO_EMBED = get_tag_embeddings(UNIQUE_TAGS)
TAG_TO_NORM_EMBED = normalize_embds(TAG_TO_EMBED)
NORM_EMBED_TO_TAG = {e: t for t, e in TAG_TO_NORM_EMBED.items()}
TAG_TO_EMBED, TAG_TO_NORM_EMBED, NORM_EMBED_TO_TAG

({'</a>': 266,
  '</abbr>': 576,
  '</address>': 911,
  '</article>': 909,
  '</aside>': 687,
  '</atom:link>': 1090,
  '</b>': 267,
  '</blockquote>': 1250,
  '</body>': 599,
  '</br>': 381,
  '</button>': 837,
  '</caption>': 919,
  '</center>': 810,
  '</channel>': 898,
  '</circle>': 795,
  '</cite>': 590,
  '</code>': 580,
  '</col>': 487,
  '</colgroup>': 1044,
  '</dc:creator>': 1178,
  '</dd>': 369,
  '</del>': 478,
  '</description>': 1357,
  '</div>': 492,
  '</dl>': 377,
  '</dt>': 385,
  '</em>': 379,
  '</fieldset>': 1017,
  '</figcaption>': 1229,
  '</figure>': 811,
  '</footer>': 824,
  '</form>': 605,
  '</guid>': 594,
  '</h1>': 322,
  '</h2>': 323,
  '</h3>': 324,
  '</h4>': 325,
  '</h5>': 326,
  '</h6>': 327,
  '</head>': 571,
  '</header>': 786,
  '</hr>': 387,
  '</html>': 606,
  '</i>': 274,
  '</iframe>': 797,
  '</img>': 486,
  '</input>': 729,
  '</ins>': 499,
  '</item>': 600,
  '</kbd>': 474,
  '</label>': 681,
  '</language>': 1005,
  '</lastbuilddate>': 15

In [16]:
fpaths,fnames = get_paths_to_files(PAGES_PATH)
tags = extract_tags_from_file(fpaths[0])
embds = tags_to_embds(tags, TAG_TO_NORM_EMBED)
back_again = embds_to_tags(embds, NORM_EMBED_TO_TAG)
print(len(tags), tags[:3])
print(len(embds), embds[:3])
print(len(back_again), back_again[:3])

1140 ['CONTENT', '<html>', 'CONTENT']
1140 [-1.1034366196023091, -1.0782869649027171, -1.1034366196023091]
1140 ['CONTENT', '<html>', 'CONTENT']


## Data Loader

In [265]:
#View a few of my examples and make sure the websites look OK going back and forth
#Load embedding data into Tensors
#Pad sequences method
#Method to view HTML string in browser
#http://stackoverflow.com/questions/36267578/how-to-display-a-python-string-as-html-in-jupyter-notebook

In [ ]:
#http://pytorch.org/docs/data.html?highlight=dataloader
#class torch.utils.data.TensorDataset(data_tensor, target_tensor)
#class torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, sampler=None, num_workers=0, collate_fn=<function default_collate>, pin_memory=False, drop_last=False)


def load_embeddings_from_dir(dir_path):
    embeddings = []